In [ ]:
forms = {
    "appeal_number": {"bench": "bench_name_1", "appeal": "app_type_1", "input": "app_number", "year": "app_year_1", "button": "b1"},
    "order_date": {"bench": "bench_name_2", "appeal": "app_type_2", "date": "order_date", "button": "b2"},
    "pron_date": {"bench": "bench_name_3", "appeal": "app_type_3", "date": "pron_date", "button": "b3"},
    "member_name": {"bench": "bench_name_4", "member": "member", "date": "order_date_2", "button": "b4"},
}


In [12]:
import os
from pydub import AudioSegment
from pydub.effects import normalize
import speech_recognition as sr

wav_path = "captcha_audio.wav"
mp3_path = "captcha_audio.mp3"

os.environ["PATH"] += os.pathsep + r"C:\Users\kaustubh.keny\ffmpeg-8.0-essentials_build\bin"

sound = AudioSegment.from_file("captcha_audio.mp3", format="mp3")
sound = sound.set_channels(1).set_frame_rate(16000)
sound = normalize(sound)
sound = sound.strip_silence(silence_len=300, silence_thresh=-40)
sound.export(wav_path, format="wav")
print(f"Converted to wav_path")

recognizer = sr.Recognizer()
with sr.AudioFile(wav_path) as source:
    audio_data = recognizer.record(source)
try:
    captcha_text = recognizer.recognize_google(audio_data).upper().replace(" ", "")
except:
    captcha_text = ""
print("Captcha:", captcha_text)

Converted to wav_path
Captcha: 7GP8A7


In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests, time, os, string, random


def generate_simple_code():
    letters = string.ascii_uppercase
    digits = string.digits

    part1 = ''.join(random.choices(letters, k=3))
    part2 = random.choice(digits) + ''.join(random.choices(letters, k=2))
    part3 = ''.join(random.choices(digits, k=2)) + random.choice(letters)

    return f"{part1}-{part2}-{part3}"


for _ in range(100):
    web_url = "https://itat.gov.in/judicial/tribunalorders"
    driver = webdriver.Chrome()
    driver.get(web_url)

    wait = WebDriverWait(driver, 10)
    accordion_btn = wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    driver.find_element(By.XPATH, "//img[@alt='Refresh Icon']").click()

    for i in range(0,5):
        
        driver.find_element(By.XPATH, "//img[@alt='Play Icon']").click()
        audio_src = driver.find_element(By.ID, "captchaAudio").get_attribute("src")
        #audio paths
        # wav_path = "audio/captcha_audio.wav"
        mp3_path = f"audio/captcha_audio_{generate_simple_code()}.mp3"

        session = requests.Session()
        for cookie in driver.get_cookies():
            session.cookies.set(cookie['name'], cookie['value'])
        response = session.get(audio_src)
        if response.status_code == 200:
            with open(mp3_path, "wb") as f:
                f.write(response.content)
            print(f"Audio file downloaded: {mp3_path}")
        print(audio_src)
        time.sleep(2)
        driver.find_element(By.XPATH, "//img[@alt='Refresh Icon']").click()
        
    driver.quit()

In [ ]:
import tkinter as tk
import webview
import threading

# Global reference to the webview window
webview_window = None

# Open website in embedded window
def open_website():
    global webview_window
    if webview_window is None:
        webview_window = webview.create_window("Embedded Site", "https://itat.gov.in/judicial/tribunalorders", width=840, height=800)
        threading.Thread(target=webview.start).start()

# Close the embedded browser
def close_website():
    global webview_window
    if webview_window:
        webview_window.destroy()
        webview_window = None

# Reload the website
def reload_website():
    if webview_window:
        webview_window.load_url("https://itat.gov.in/judicial/tribunalorders")

# GUI setup
root = tk.Tk()
root.title("Web Controller")
root.geometry("1200x800")
root.configure(bg="#2e2e2e")  # Dark grey

# Left panel (30%)
left_frame = tk.Frame(root, bg="#2e2e2e", width=360, height=800)
left_frame.place(x=0, y=0)

# Buttons
btn_open = tk.Button(left_frame, text="Open Website", command=open_website, bg="#444", fg="white", width=20)
btn_open.pack(pady=20)

btn_close = tk.Button(left_frame, text="Close Website", command=close_website, bg="#444", fg="white", width=20)
btn_close.pack(pady=20)

btn_reload = tk.Button(left_frame, text="Reload Website", command=reload_website, bg="#444", fg="white", width=20)
btn_reload.pack(pady=20)

# Right panel (70%) — placeholder for embedded browser
right_frame = tk.Frame(root, bg="#1e1e1e", width=840, height=800)
right_frame.place(x=360, y=0)

root.mainloop()

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests, time, os
from pydub import AudioSegment
import speech_recognition as sr
from pydub.effects import normalize
import pandas as pd

def run_program(url,date,bench = 1, appeal = 1):
    driver = webdriver.Chrome()
    driver.get(url)

    wait = WebDriverWait(driver, 10)

    accordion_btn = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#headingTwo button")))
    driver.execute_script("arguments[0].click();", accordion_btn)
    time.sleep(1)


    benchSelect = Select(driver.find_element(By.ID, "bench_name_2"))
    benchSelect.select_by_index(bench)
    # benchSelect.select_by_visible_text("Mumbai")
    bench_select = benchSelect.first_selected_option.text
    
    time.sleep(1)
    benchAppeal = Select(driver.find_element(By.ID, "app_type_2"))
    benchAppeal.select_by_index(appeal)
    bench_appeal = benchAppeal.first_selected_option.text

    time.sleep(1)

    order_date_input = driver.find_element(By.ID, "order_date")
    driver.execute_script("arguments[0].removeAttribute('readonly')", order_date_input)
    driver.execute_script("arguments[0].value = arguments[1];", order_date_input, date)
    driver.execute_script("arguments[0].dispatchEvent(new Event('change'));", order_date_input)
    time.sleep(1)


    driver.find_element(By.XPATH, "//img[@alt='Play Icon']").click()
    time.sleep(2)
    audio_src = driver.find_element(By.ID, "captchaAudio").get_attribute("src")


    #audio paths
    wav_path = "captcha_audio.wav"
    mp3_path = "captcha_audio.mp3"
    
    #selectors
    # "td:nth-child(2)", ".text-center:nth-child(4) a"

    session = requests.Session()
    for cookie in driver.get_cookies():
        session.cookies.set(cookie['name'], cookie['value'])
    response = session.get(audio_src)
    with open(mp3_path, "wb") as f:
        f.write(response.content)
    
    #mp3 -> wav
    os.environ["PATH"] += os.pathsep + r"C:\Users\kaustubh.keny\ffmpeg-8.0-essentials_build\bin"
    sound = AudioSegment.from_file("captcha_audio.mp3", format="mp3")
    sound = sound.set_channels(1).set_frame_rate(16000)
    sound = normalize(sound)
    sound = sound.strip_silence(silence_len=300, silence_thresh=-40)
    sound.export(wav_path, format="wav")


    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_path) as source:
        audio_data = recognizer.record(source)
    try:
        captcha_text = recognizer.recognize_google(audio_data).upper().replace(" ", "")
    except:
        captcha_text = ""
    print("Captcha:", captcha_text)


    captcha_input = driver.find_element(By.ID, "captcha")
    for char in captcha_text:
        captcha_input.send_keys(char)
        time.sleep(0.2)
        
    input()
    driver.find_element(By.ID, "b2").click()
    time.sleep(.5)

    
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#results")))
    
    no_data_cells = driver.find_elements(
    By.CSS_SELECTOR,
    "#results table tbody tr td[colspan='5']"
    )

    data = []
    if no_data_cells and "No Records Found" in no_data_cells[0].text:
        print("No records found. Skipping pagination.")
    else:
        data = []
        for page_num in range(1, 6):
            try:
                page_button = driver.find_element(By.XPATH, f"//input[@name='btnPage' and @value='{page_num}']")
                driver.execute_script("arguments[0].click();", page_button)

                print(f"Clicked page {page_num}")
                time.sleep(2)            
                rows = driver.find_elements(By.CSS_SELECTOR, "#results table tbody tr")
                for row in rows:
                    parties = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                    order_link = row.find_element(By.CSS_SELECTOR, ".text-center:nth-child(4) a").get_attribute("href")
                    data.append([parties, order_link])
                        
            except Exception as e:
                print(f"Failed to click page {page_num}: {e}")


    if data:
        df = pd.DataFrame(data,columns=["Bench Name","Order Link"])
        
        df["Order Link"] = df["Order Link"].apply(
            lambda url: f'=HYPERLINK("{url}", "{url}")'
        )

        df.to_excel(f"{bench_select}-{bench_appeal}.xlsx".lower())
    else:
        print("Empty Data Content.")
    
    driver.quit()


KeyboardInterrupt: 

In [10]:
web_url = "https://itat.gov.in/judicial/tribunalorders"


run_program(web_url,"18/06/2025",11,1)
# run_program(web_url,"11/09/2025",11,2)

Captcha: 24KFD
Clicked page 1
Clicked page 2
Clicked page 3
Clicked page 4
Clicked page 5


APPEAL OPTIONS

In [50]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://itat.gov.in/judicial/tribunalorders")

wait = WebDriverWait(driver, 10)
accordion_btn = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#headingTwo .accordion-button")))
driver.execute_script("arguments[0].click();", accordion_btn)


wait.until(EC.presence_of_element_located((By.ID, "bench_name_2")))


bench_select = Select(driver.find_element(By.ID, "bench_name_2"))
# bench_options = [opt.get_attribute("value") for opt in bench_select.options if opt.get_attribute("value")]
bench_options = [(opt.get_attribute("value"), opt.text.strip()) for opt in bench_select.options if opt.get_attribute("value")]


appeal_select = Select(driver.find_element(By.ID, "app_type_2"))
# appeal_options = [opt.get_attribute("value") for opt in appeal_select.options if opt.get_attribute("value")]
appeal_options = [(opt.get_attribute("value"), opt.text.strip()) for opt in appeal_select.options if opt.get_attribute("value")]
print("Bench Options:", bench_options)
print("Appeal Options:", appeal_options)

driver.quit()


Bench Options: [('203', 'Agra'), ('205', 'Ahmedabad'), ('207', 'Allahabad'), ('209', 'Amritsar'), ('211', 'Bangalore'), ('215', 'Chandigarh'), ('217', 'Chennai'), ('219', 'Cochin'), ('221', 'Cuttack'), ('260', 'Dehradun'), ('201', 'Delhi'), ('223', 'Guwahati'), ('225', 'Hyderabad'), ('227', 'Indore'), ('229', 'Jabalpur'), ('231', 'Jaipur'), ('233', 'Jodhpur'), ('235', 'Kolkata'), ('237', 'Lucknow'), ('199', 'Mumbai'), ('239', 'Nagpur'), ('241', 'Panaji'), ('243', 'Patna'), ('245', 'Pune'), ('247', 'Raipur'), ('249', 'Rajkot'), ('251', 'Ranchi'), ('256', 'Surat'), ('258', 'Varanasi'), ('253', 'Visakhapatnam')]
Appeal Options: [('ITA', 'Income Tax Appeal'), ('CO', 'Cross Objection'), ('ITSSA', 'Income Tax (Search & Seizure) Appeal'), ('ITTPA', 'Income Tax (Transfer Pricing) Appeal'), ('ITITA', 'Income Tax (International Taxation) Appeal'), ('WTA', 'Wealth Tax Appeal'), ('BMA', 'Black Money Appeal'), ('EDA', 'Estate Duty Appeal'), ('INTTA', 'Interest Tax Appeal'), ('GTA', 'Gift Tax Appeal

In [3]:
# List of bench cities
bench_cities = [
    "Agra", "Ahmedabad", "Allahabad", "Amritsar", "Bangalore", "Chandigarh", "Chennai",
    "Cochin", "Cuttack", "Dehradun", "Delhi", "Guwahati", "Hyderabad", "Indore", "Jabalpur",
    "Jaipur", "Jodhpur", "Kolkata", "Lucknow", "Mumbai", "Nagpur", "Panaji", "Patna", "Pune",
    "Raipur", "Rajkot", "Ranchi", "Surat", "Varanasi", "Visakhapatnam"
]

# List of appeal types (human-readable)
appeal_types = [
    "Income Tax Appeal", "Cross Objection", "Income Tax (Search & Seizure) Appeal",
    "Income Tax (Transfer Pricing) Appeal", "Income Tax (International Taxation) Appeal",
    "Wealth Tax Appeal", "Black Money Appeal", "Estate Duty Appeal", "Interest Tax Appeal",
    "Gift Tax Appeal", "TDS Appeal", "Security Transaction Tax Appeal", "Expenditure Tax Appeal",
    "Sur Tax Appeal", "High Court Decision", "Stay Application", "Miscellaneous Application",
    "Reference Application"
]

# Generate the dictionary
# city_appeals = {city: appeal_types for city in bench_cities}

# # Optional: print a sample
# for city, appeals in city_appeals.items():
#     print(f"{city}:")
#     for appeal in appeals:
#         print(f"  - {appeal}")
#     print()


data = [[bench,appeal] for bench in bench_cities for appeal in appeal_types]

In [ ]:
import os
import wave
from vosk import Model, KaldiRecognizer

AUDIO_FILE = "cleaned.wav"
MODEL_DIR = r"C:\Users\kaustubh.keny\vosk\vosk-model-small-en-us-0.15"
MODEL_DIR_LARGE = r"C:\Users\kaustubh.keny\vosk\vosk-model-en-us-0.22"


with wave.open(AUDIO_FILE, "rb") as wf:
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() not in [8000, 16000]:
        raise ValueError("Audio must be mono PCM with sample rate 8000 or 16000 Hz")

    # Load Vosk model
    model_path = os.path.abspath(MODEL_DIR)
    model = Model(model_path)
    recognizer = KaldiRecognizer(model, wf.getframerate())

    print("Transcription:")
    while True:
        frames = wf.readframes(2000)
        if not frames:
            break
        if recognizer.AcceptWaveform(frames):
            result = recognizer.Result()
            print(result)
        else:
            print(recognizer.PartialResult())